In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [30]:
BOTCONFIG_PATH = r'C:\Users\SIDDHARTHABHATTACHAR\Documents\Copy\MachineLearning\sampleProjects\botforbot\config\bot_booking.json'

In [31]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open(BOTCONFIG_PATH).read()
botdata = json.loads(data_file)
intent = botdata['intents']

In [32]:
intent

[{'patterns': ['I want to book a cab',
   'book my cab',
   'get me taxi',
   'book a car'],
  'journeyName': 'bookcab',
  'inputMapping': {'data': 'message',
   'channel': 'cabbookchannel',
   'user': 'Sidd'}},
 {'patterns': ['I want to book a hotel', 'book my hotel'],
  'journeyName': 'bookhotel',
  'inputMapping': {'data': 'message',
   'channel': 'hostelbookchannel',
   'user': 'Sidd'}}]

In [33]:
for pattern in intent:
    for sentence in pattern['patterns']:
        w = nltk.word_tokenize(sentence)
        words.extend(w)
        documents.append((w, pattern['journeyName']))
        if pattern['journeyName'] not in classes:
            classes.append(pattern['journeyName'])
print("words >> ", words, "documents >> ", documents, "classes >> ", classes)

words >>  ['I', 'want', 'to', 'book', 'a', 'cab', 'book', 'my', 'cab', 'get', 'me', 'taxi', 'book', 'a', 'car', 'I', 'want', 'to', 'book', 'a', 'hotel', 'book', 'my', 'hotel'] documents >>  [(['I', 'want', 'to', 'book', 'a', 'cab'], 'bookcab'), (['book', 'my', 'cab'], 'bookcab'), (['get', 'me', 'taxi'], 'bookcab'), (['book', 'a', 'car'], 'bookcab'), (['I', 'want', 'to', 'book', 'a', 'hotel'], 'bookhotel'), (['book', 'my', 'hotel'], 'bookhotel')] classes >>  ['bookcab', 'bookhotel']


In [34]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
print("words >> ", words)

words >>  ['a', 'book', 'cab', 'car', 'get', 'hotel', 'i', 'me', 'my', 'taxi', 'to', 'want']


In [35]:
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

6 documents
2 classes ['bookcab', 'bookhotel']
12 unique lemmatized words ['a', 'book', 'cab', 'car', 'get', 'hotel', 'i', 'me', 'my', 'taxi', 'to', 'want']


In [36]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [37]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

In [38]:
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
print ("training >> ", training)

training >>  [[[1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1], [1, 0]], [[0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0], [1, 0]], [[0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0], [1, 0]], [[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0]], [[1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1], [0, 1]], [[0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0], [0, 1]]]


In [39]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


C:\Users\SIDDHARTHABHATTACHAR\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until


In [43]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [44]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [46]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('primaryintent_model.h5', hist)

Train on 6 samples
Epoch 1/200
6/6 [==============================] - 0s 4ms/sample - loss: 0.0257 - accuracy: 1.0000
Epoch 2/200
6/6 [==============================] - 0s 2ms/sample - loss: 4.8377e-05 - accuracy: 1.0000
Epoch 3/200
6/6 [==============================] - 0s 2ms/sample - loss: 0.0021 - accuracy: 1.0000
Epoch 4/200
6/6 [==============================] - 0s 4ms/sample - loss: 4.4304e-05 - accuracy: 1.0000
Epoch 5/200
6/6 [==============================] - 0s 1ms/sample - loss: 0.0016 - accuracy: 1.0000
Epoch 6/200
6/6 [==============================] - 0s 2ms/sample - loss: 3.0544e-04 - accuracy: 1.0000
Epoch 7/200
6/6 [==============================] - 0s 1ms/sample - loss: 0.0277 - accuracy: 1.0000
Epoch 8/200
6/6 [==============================] - 0s 1ms/sample - loss: 4.4795e-04 - accuracy: 1.0000
Epoch 9/200
6/6 [==============================] - 0s 1ms/sample - loss: 9.7284e-05 - accuracy: 1.0000
Epoch 10/200
6/6 [==============================] - 0s 2ms/sample - lo